# Agregarea Datelor

In [1]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os

### Ne conectam la Neon

In [2]:
load_dotenv()
engine = create_engine(os.getenv("NEON_URL"))


In [3]:
tables = pd.read_sql("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public';
""", engine)

tables

,table_name
0,eia
1,wdi
2,all_data_long


# Incărcăm tabelele

In [4]:
wdi = pd.read_sql("SELECT * FROM wdi;", engine)
wdi.head(2)

,country_iso3,year,indicator_code,indicator_name,indicator_label,value
0,AFG,2023,NY.GDP.PCAP.CD,gdp_per_capita,GDP per capita (current US$),413.757895
1,AFG,2022,NY.GDP.PCAP.CD,gdp_per_capita,GDP per capita (current US$),357.261153


In [5]:
eia = pd.read_sql("SELECT * FROM eia;", engine)
eia.head(2)

,country_iso3,country_name,year,indicator_code,indicator_name,indicator_label,unit,unit_name,value
0,ABW,Aruba,2024,GROS_GDP,gross_domestic_product_gdp,Gross Domestic Product – Gdp,BDOLPPP,billion_dollars_at_purchasing_power_parities,3.768053
1,AFG,Afghanistan,2024,GROS_GDP,gross_domestic_product_gdp,Gross Domestic Product – Gdp,BDOLPPP,billion_dollars_at_purchasing_power_parities,64.92137


### Structura datelor

In [6]:
wdi.info()
print(wdi["country_iso3"].nunique(), wdi["year"].min(), wdi["year"].max())
wdi["indicator_code"].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62208 entries, 0 to 62207
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country_iso3     62208 non-null  object 
 1   year             62208 non-null  int64  
 2   indicator_code   62208 non-null  object 
 3   indicator_name   62208 non-null  object 
 4   indicator_label  62208 non-null  object 
 5   value            57302 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 2.8+ MB
216 2000 2023


indicator_code
NY.GDP.PCAP.CD       5184
SP.POP.TOTL          5184
SP.DYN.LE00.IN       5184
RQ.EST               5184
SP.URB.TOTL.IN.ZS    5184
EG.USE.PCAP.KG.OE    5184
EG.FEC.RNEW.ZS       5184
EG.ELC.ACCS.ZS       5184
GE.EST               5184
RL.EST               5184
CC.EST               5184
IT.CEL.SETS.P2       5184
Name: count, dtype: int64

In [17]:
eia.info()
print(eia["country_iso3"].nunique(), eia["year"].min(), eia["year"].max())
#eia["indicator_code"].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299340 entries, 0 to 299339
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   country_iso3     299340 non-null  object
 1   country_name     299340 non-null  object
 2   year             299340 non-null  int64 
 3   indicator_code   299340 non-null  object
 4   indicator_name   299340 non-null  object
 5   indicator_label  299340 non-null  object
 6   unit             299340 non-null  object
 7   unit_name        299340 non-null  object
 8   value            297916 non-null  object
dtypes: int64(1), object(8)
memory usage: 20.6+ MB
233 2001 2024


In [13]:
eia.head(2)


,country_iso3,country_name,year,indicator_code,indicator_name,indicator_label,unit,unit_name,value
0,ABW,Aruba,2024,GROS_GDP,gross_domestic_product_gdp,Gross Domestic Product – Gdp,BDOLPPP,billion_dollars_at_purchasing_power_parities,3.768053
1,AFG,Afghanistan,2024,GROS_GDP,gross_domestic_product_gdp,Gross Domestic Product – Gdp,BDOLPPP,billion_dollars_at_purchasing_power_parities,64.92137


In [14]:
wdi.head(2)

,country_iso3,year,indicator_code,indicator_name,indicator_label,value
0,AFG,2023,NY.GDP.PCAP.CD,gdp_per_capita,GDP per capita (current US$),413.757895
1,AFG,2022,NY.GDP.PCAP.CD,gdp_per_capita,GDP per capita (current US$),357.261153


In [15]:
eia.columns

Index(['country_iso3', 'country_name', 'year', 'indicator_code',
       'indicator_name', 'indicator_label', 'unit', 'unit_name', 'value'],
      dtype='object')

In [16]:
wdi.columns

Index(['country_iso3', 'year', 'indicator_code', 'indicator_name',
       'indicator_label', 'value'],
      dtype='object')

### Pregatim wdi

#### 1. Country name

In [7]:
# mapping din EIA: ISO → nume țară
iso_to_name = (
    eia[["country_iso3", "country_name"]]
    .drop_duplicates()
    .set_index("country_iso3")["country_name"]
    .to_dict()
)

# adăugăm coloana în WDI
wdi["country_name"] = wdi["country_iso3"].map(iso_to_name)

In [8]:
missing = wdi[wdi["country_name"].isna()]
missing.country_iso3.value_counts()


country_iso3
AND    288
CUW    288
IMN    288
XKX    288
LIE    288
MHL    288
MCO    288
PLW    288
SMR    288
SXM    288
MAF    288
Name: count, dtype: int64

In [9]:
remove_iso = set(missing.country_iso3)
wdi = wdi[~wdi["country_iso3"].isin(remove_iso)].copy()
wdi.country_name.isna().sum()

np.int64(0)

#### 2. Unit

In [55]:
wdi[["indicator_code","indicator_label"]].drop_duplicates().sort_values("indicator_code")


,indicator_code,indicator_label
51840,CC.EST,Control of Corruption: Estimate
31296,EG.ELC.ACCS.ZS,Access to electricity (% of population)
26102,EG.FEC.RNEW.ZS,Renewable energy consumption (% of total final...
20897,EG.USE.PCAP.KG.OE,Energy use (kg of oil equivalent per capita)
36558,GE.EST,Government Effectiveness: Estimate
57024,IT.CEL.SETS.P2,Mobile cellular subscriptions (per 100 people)
0,NY.GDP.PCAP.CD,GDP per capita (current US$)
46656,RL.EST,Rule of Law: Estimate
10441,RQ.EST,Regulatory Quality: Estimate
10368,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)"


In [10]:
wdi_units = {
    "NY.GDP.PCAP.CD": "current US$",
    "SP.POP.TOTL": "persons",
    "SP.DYN.LE00.IN": "years",
    "RQ.EST": "index",
    "SP.URB.TOTL.IN.ZS": "percent",
    "EG.USE.PCAP.KG.OE": "kg of oil equivalent per capita",
    "EG.FEC.RNEW.ZS": "percent",
    "EG.ELC.ACCS.ZS": "percent",
    "GE.EST": "index",
    "RL.EST": "index",
    "CC.EST": "index",
    "IT.CEL.SETS.P2": "per 100 people"
}


In [11]:
# creem coloana unit name pe baza dictionarului
wdi["unit_name"] = wdi["indicator_code"].map(wdi_units)
wdi["unit_name"].isna().sum()


np.int64(0)

# Eia units

In [12]:
pd.set_option('display.max_rows', None)

all_indicators = eia[["indicator_name", "unit_name"]].drop_duplicates()
# all_indicators are coloane: indicator_name, unit_name
# Vrem pentru fiecare indicator lista de unități distincte

units_per_indicator = (
    all_indicators
    .groupby("indicator_name")["unit_name"]
    .unique()
)
units_per_indicator 

indicator_name
co2_emissions_emissions                       [million_metric_tonnes_carbon_dioxide]
coal_consumption                   [1000_metric_tons, million_metric_tons_of_oil_...
coal_exports                       [1000_metric_tons, million_metric_tons_of_oil_...
coal_generation                                              [billion_kilowatthours]
coal_imports                       [1000_metric_tons, million_metric_tons_of_oil_...
coal_production                    [quadrillion_btu, terajoules, thousand_short_t...
coal_reserves                                                   [million_short_tons]
electricity_capacity                                             [million_kilowatts]
electricity_consumption                                      [billion_kilowatthours]
electricity_distribution_losses                              [billion_kilowatthours]
electricity_exports                [billion_kilowatthours, million_metric_tons_of...
electricity_generation                            

In [13]:
# Selectăm doar indicatorii care au mai multe unități
multi_unit_indicators = units_per_indicator[units_per_indicator.apply(len) > 1]

# Print frumos
for ind, units in multi_unit_indicators.items():
    print(f"$ {ind}:")
    for u in units:
        print(f"   • {u}")

$ coal_consumption:
   • 1000_metric_tons
   • million_metric_tons_of_oil_equivalent
   • quadrillion_btu
   • terajoules
   • thousand_short_tons
$ coal_exports:
   • 1000_metric_tons
   • million_metric_tons_of_oil_equivalent
   • quadrillion_btu
   • terajoules
   • thousand_short_tons
$ coal_imports:
   • 1000_metric_tons
   • million_metric_tons_of_oil_equivalent
   • quadrillion_btu
   • terajoules
   • thousand_short_tons
$ coal_production:
   • quadrillion_btu
   • terajoules
   • thousand_short_tons
   • 1000_metric_tons
   • million_metric_tons_of_oil_equivalent
$ electricity_exports:
   • billion_kilowatthours
   • million_metric_tons_of_oil_equivalent
   • quadrillion_btu
   • terajoules
$ electricity_imports:
   • terajoules
   • billion_kilowatthours
   • million_metric_tons_of_oil_equivalent
   • quadrillion_btu
$ electricity_net_imports:
   • billion_kilowatthours
   • million_metric_tons_of_oil_equivalent
   • quadrillion_btu
   • terajoules
$ primary_energy_consumptio

In [ ]:
# 1. unitățile standard
preferred_units = {
    "coal_generation": "billion_kilowatthours",
    "oil_generation": "billion_kilowatthours",
    "natural_gas_generation": "billion_kilowatthours",
    "electricity_generation": "billion_kilowatthours",
    "electricity_consumption": "billion_kilowatthours",
    "electricity_exports": "billion_kilowatthours",
    "electricity_imports": "billion_kilowatthours",
    "electricity_net_imports": "billion_kilowatthours",
    "renewables_generation": "billion_kilowatthours",

    "primary_energy_consumption": "million_metric_tons_of_oil_equivalent",
    "primary_energy_production": "million_metric_tons_of_oil_equivalent",

    "coal_consumption": "1000_metric_tons",
    "coal_production": "1000_metric_tons",
    "coal_imports": "1000_metric_tons",
    "coal_exports": "1000_metric_tons",
}

# 2. tabel unic de conversie
CONVERSION = {
    ("billion_kilowatthours", "billion_kilowatthours"): 1,
    ("billion_kilowatthours", "million_metric_tons_of_oil_equivalent"): 11.63,
    ("billion_kilowatthours", "quadrillion_btu"): 293.071,
    ("billion_kilowatthours", "terajoules"): 0.000277778,

    ("million_metric_tons_of_oil_equivalent", "million_metric_tons_of_oil_equivalent"): 1,
    ("million_metric_tons_of_oil_equivalent", "quadrillion_btu"): 25.2,
    ("million_metric_tons_of_oil_equivalent", "terajoules"): 0.00002388,

    ("1000_metric_tons", "1000_metric_tons"): 1,
    ("1000_metric_tons", "thousand_short_tons"): 0.907185,
}



In [38]:
# -------------------------------------------------------------
eia_f = eia.copy()

for ind, unit in preferred_units.items():
    # pentru indicatorul ind, păstrăm doar rândurile cu unit=preferată
    eia_f = eia_f[
        ~(
            (eia_f["indicator_name"] == ind) &
            (eia_f["unit_name"] != unit)
        )
    ]

# -------------------------------------------------------------
# 3. Pentru ceilalți indicatori → păstrăm doar o unitate (prima)
# -------------------------------------------------------------
other_inds = set(eia_f["indicator_name"]) - set(preferred_units.keys())

rows = []
for ind in other_inds:
    sub = eia_f[eia_f["indicator_name"] == ind]
    first_unit = sub["unit_name"].iloc[0]
    rows.append(sub[sub["unit_name"] == first_unit])

eia_f2 = pd.concat(rows, ignore_index=True)

# Adăugăm și indicatorii standardizați deja
eia_std = pd.concat([
    eia_f2,
    eia_f[eia_f["indicator_name"].isin(preferred_units.keys())]
], ignore_index=True)

# -------------------------------------------------------------
# 4. Test: verificăm dacă fiecare indicator are o singură unitate
# -------------------------------------------------------------
print(
    eia_std.groupby("indicator_name")["unit_name"].nunique().sort_values(ascending=False)
)

indicator_name
co2_emissions_emissions            1
coal_consumption                   1
coal_exports                       1
coal_generation                    1
coal_imports                       1
coal_production                    1
coal_reserves                      1
electricity_capacity               1
electricity_consumption            1
electricity_distribution_losses    1
electricity_exports                1
electricity_generation             1
electricity_imports                1
electricity_net_imports            1
gross_domestic_product_gdp         1
natural_gas_consumption            1
natural_gas_emissions              1
natural_gas_generation             1
natural_gas_production             1
oil_generation                     1
population                         1
primary_energy_consumption         1
primary_energy_production          1
renewables_capacity                1
renewables_generation              1
Name: unit_name, dtype: int64


In [39]:
eia_std.unit_name.value_counts()

unit_name
billion_kilowatthours                           51928
1000_metric_tons                                21380
million_metric_tons_of_oil_equivalent           10674
quadrillion_btu                                 10668
million_metric_tonnes_carbon_dioxide            10589
million_kilowatts                               10354
people_in_thousands                              5519
billion_dollars_at_purchasing_power_parities     5507
million_short_tons                               3626
Name: count, dtype: int64

In [41]:
eia_std.groupby("indicator_name")["unit_name"].unique()


indicator_name
co2_emissions_emissions                    [million_metric_tonnes_carbon_dioxide]
coal_consumption                                               [1000_metric_tons]
coal_exports                                                   [1000_metric_tons]
coal_generation                                           [billion_kilowatthours]
coal_imports                                                   [1000_metric_tons]
coal_production                                                [1000_metric_tons]
coal_reserves                                                [million_short_tons]
electricity_capacity                                          [million_kilowatts]
electricity_consumption                                   [billion_kilowatthours]
electricity_distribution_losses                           [billion_kilowatthours]
electricity_exports                                       [billion_kilowatthours]
electricity_generation                                    [billion_kilowatthours]
e

In [42]:
for ind in preferred_units:
    print(ind, "→", eia_f[eia_f["indicator_name"] == ind]["unit_name"].unique())


coal_generation → ['billion_kilowatthours']
oil_generation → ['billion_kilowatthours']
natural_gas_generation → ['billion_kilowatthours']
electricity_generation → ['billion_kilowatthours']
electricity_consumption → ['billion_kilowatthours']
electricity_exports → ['billion_kilowatthours']
electricity_imports → ['billion_kilowatthours']
electricity_net_imports → ['billion_kilowatthours']
renewables_generation → ['billion_kilowatthours']
primary_energy_consumption → ['million_metric_tons_of_oil_equivalent']
primary_energy_production → ['million_metric_tons_of_oil_equivalent']
coal_consumption → ['1000_metric_tons']
coal_production → ['1000_metric_tons']
coal_imports → ['1000_metric_tons']
coal_exports → ['1000_metric_tons']


### Unim tabelele

In [58]:
wdi.columns

Index(['country_iso3', 'year', 'indicator_code', 'indicator_name',
       'indicator_label', 'value', 'country_name', 'unit_name'],
      dtype='object')

In [43]:
eia_std.columns


Index(['country_iso3', 'country_name', 'year', 'indicator_code',
       'indicator_name', 'indicator_label', 'unit', 'unit_name', 'value'],
      dtype='object')

#### Schema finala

country_iso3  
country_name  
year  
indicator_name  
unit_name  
value


In [44]:
# creem wdi final 
wdi_final = wdi[[
    "country_iso3",
    "country_name",
    "year",
    "indicator_name",
    "unit_name",
    "value"
]].copy()

# creem eia final
eia_final = eia_std[[
    "country_iso3",
    "country_name",
    "year",
    "indicator_name",
    "unit_name",
    "value"
]].copy()

eia_final["value"] = pd.to_numeric(eia_final["value"], errors="coerce")

#### Concat

In [45]:
all_data_long = pd.concat([wdi_final, eia_final], ignore_index=True)


In [46]:
all_data_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189285 entries, 0 to 189284
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   country_iso3    189285 non-null  object 
 1   country_name    189285 non-null  object 
 2   year            189285 non-null  int64  
 3   indicator_name  189285 non-null  object 
 4   unit_name       189285 non-null  object 
 5   value           178737 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 8.7+ MB


In [47]:
all_data_long.rename(columns={"country_iso3":"country"}, inplace = True)

In [48]:
all_data_long.head()

,country,country_name,year,indicator_name,unit_name,value
0,AFG,Afghanistan,2023,gdp_per_capita,current US$,413.757895
1,AFG,Afghanistan,2022,gdp_per_capita,current US$,357.261153
2,AFG,Afghanistan,2021,gdp_per_capita,current US$,356.496214
3,AFG,Afghanistan,2020,gdp_per_capita,current US$,510.787063
4,AFG,Afghanistan,2019,gdp_per_capita,current US$,496.602504


In [49]:
all_data_long.duplicated().sum()


np.int64(0)

In [50]:
all_data_long["unit_name"].value_counts()

unit_name
billion_kilowatthours                           51928
1000_metric_tons                                21380
index                                           19680
percent                                         14760
million_metric_tons_of_oil_equivalent           10674
quadrillion_btu                                 10668
million_metric_tonnes_carbon_dioxide            10589
million_kilowatts                               10354
people_in_thousands                              5519
billion_dollars_at_purchasing_power_parities     5507
current US$                                      4920
per 100 people                                   4920
kg of oil equivalent per capita                  4920
persons                                          4920
years                                            4920
million_short_tons                               3626
Name: count, dtype: int64

# Salvam si incarcam in baza de date

In [51]:
all_data_long.to_csv("../data/all_data_long.csv",index = False)

In [52]:
load_dotenv()
engine = create_engine(os.getenv("NEON_URL"))

In [53]:
all_data_long.to_sql(
    "all_data_long",
    engine,
    if_exists="replace",
    index=False
)


285

In [54]:
pd.read_sql("SELECT COUNT(*) FROM all_data_long;", engine)

,count
0,189285
